In [1]:
import pandas as pd
import numpy as np
import altair as alt
import time
import requests

In [2]:
data = pd.read_csv("../../data/raw/environment data.csv")

In [3]:
display(data.shape)
data.head()

(1318, 94)

,STATION,NAME,LATITUDE,LONGITUDE,ELEVATION,DATE,AWND,AWND_ATTRIBUTES,CDSD,CDSD_ATTRIBUTES,...,WSFG,WSFG_ATTRIBUTES,,,.1,.1,.2,.2,.3,Unnamed: 93
0,USC00437612,"SOUTH LINCOLN, VT US",44.0725,-72.9736,408.7,1990,,,,,...,,,,,,,,,,NaN
1,USC00437612,"SOUTH LINCOLN, VT US",44.0725,-72.9736,408.7,1991,,,,,...,,,,,,,,,,NaN
2,USC00437612,"SOUTH LINCOLN, VT US",44.0725,-72.9736,408.7,1992,,,66,,...,,,,,,,,,,NaN
3,USC00437612,"SOUTH LINCOLN, VT US",44.0725,-72.9736,408.7,1993,,,142,,...,,,,,,,,,,NaN
4,USC00437612,"SOUTH LINCOLN, VT US",44.0725,-72.9736,408.7,1994,,,144,,...,,,,,,,,,,NaN


In [4]:
data.rename(columns=lambda x: x.strip(), inplace=True)

- STATION (11 characters) is the station identification code. 
- STATION_NAME (max 50 characters) is the name of the station (usually city/airport name). This is an optional output field.
- LATITUDE (8 characters) is the latitude (decimated degrees w/Northern Hemisphere values > 0). This is an optional output field.
- LONGITUDE (9 characters) is the longitude (decimated degrees w/Western Hemisphere values < 0 and Eastern Hemisphere values > 0). This is an optional output field.
- ELEVATION (13 characters) is the elevation above mean sea level in meters (to nearest thousandth of a meter). This is an optional output field.
- DATE is the year of the record (4 digits). 


In [5]:
new_data = data.copy(deep=True)
new_data.insert(4, "COUNTY", None, True)
new_data.head()

,STATION,NAME,LATITUDE,LONGITUDE,COUNTY,ELEVATION,DATE,AWND,AWND_ATTRIBUTES,CDSD,...,WSFG,WSFG_ATTRIBUTES,,,.1,.1,.2,.2,.3,Unnamed: 93
0,USC00437612,"SOUTH LINCOLN, VT US",44.0725,-72.9736,None,408.7,1990,,,,...,,,,,,,,,,NaN
1,USC00437612,"SOUTH LINCOLN, VT US",44.0725,-72.9736,None,408.7,1991,,,,...,,,,,,,,,,NaN
2,USC00437612,"SOUTH LINCOLN, VT US",44.0725,-72.9736,None,408.7,1992,,,66,...,,,,,,,,,,NaN
3,USC00437612,"SOUTH LINCOLN, VT US",44.0725,-72.9736,None,408.7,1993,,,142,...,,,,,,,,,,NaN
4,USC00437612,"SOUTH LINCOLN, VT US",44.0725,-72.9736,None,408.7,1994,,,144,...,,,,,,,,,,NaN


In [6]:
stations = new_data.STATION.unique()
start = time.time()
for station in stations:
    latitude, longitude = new_data[new_data['STATION'] == station].values[0][2:4]
    payload = { 'latitude': latitude, 'longitude': longitude, 'format': 'json' }
    r = requests.get('https://geo.fcc.gov/api/census/area', params=payload).json()
    county = r['County']['name']
    new_data.loc[new_data['STATION'] == station, 'COUNTY'] = county
display(time.time() - start)

16.88070797920227

In [7]:
display(new_data)

,STATION,NAME,LATITUDE,LONGITUDE,COUNTY,ELEVATION,DATE,AWND,AWND_ATTRIBUTES,CDSD,...,WSFG,WSFG_ATTRIBUTES,,,.1,.1,.2,.2,.3,Unnamed: 93
0,USC00437612,"SOUTH LINCOLN, VT US",44.0725,-72.9736,Addison,408.7,1990,,,,...,,,,,,,,,,NaN
1,USC00437612,"SOUTH LINCOLN, VT US",44.0725,-72.9736,Addison,408.7,1991,,,,...,,,,,,,,,,NaN
2,USC00437612,"SOUTH LINCOLN, VT US",44.0725,-72.9736,Addison,408.7,1992,,,66,...,,,,,,,,,,NaN
3,USC00437612,"SOUTH LINCOLN, VT US",44.0725,-72.9736,Addison,408.7,1993,,,142,...,,,,,,,,,,NaN
4,USC00437612,"SOUTH LINCOLN, VT US",44.0725,-72.9736,Addison,408.7,1994,,,144,...,,,,,,,,,,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1313,USC00438169,"SUTTON, VT US",44.6122,-72.0481,Caledonia,457.2,2015,,,201,...,,,,,,,,,,NaN
1314,USC00438169,"SUTTON, VT US",44.6122,-72.0481,Caledonia,457.2,2016,,,247,...,,,,,,,,,,NaN
1315,USC00438169,"SUTTON, VT US",44.6122,-72.0481,Caledonia,457.2,2017,,,195,...,,,,,,,,,,NaN
1316,USC00438169,"SUTTON, VT US",44.6122,-72.0481,Caledonia,457.2,2018,,,,...,,,,,,,,,,NaN


In [8]:
new_data.to_csv("../../data/raw/environment data - with counties.csv", index=False)